# 经销商订单执行物流最优成本规划-下

In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
data = pd.read_csv('C:/ML/path-no-D2-D4.csv')

In [3]:
import networkx as nx

FG = nx.DiGraph()
FG = nx.from_pandas_edgelist(data, source='origin', target='dest', edge_attr=True,create_using=nx.DiGraph())

FG.nodes()
FG.edges()

OutEdgeView([('F1', 'D1'), ('F1', 'D2'), ('F1', 'D3'), ('F1', 'D4'), ('F1', 'C1'), ('F1', 'C3'), ('F1', 'C4'), ('F1', 'C6'), ('D1', 'C2'), ('D1', 'C3'), ('D1', 'C4'), ('D1', 'C6'), ('D3', 'C2'), ('D3', 'C3'), ('D3', 'C5'), ('D3', 'C6'), ('F2', 'D2'), ('F2', 'D3'), ('F2', 'D4'), ('F2', 'C1')])

In [4]:
#若从F1工厂发出
path1={}
for i in range(1,7):
    path1['C'+str(i)]=nx.dijkstra_path(FG, source='F1', target="C"+str(i), weight='cost')
path1

{'C1': ['F1', 'C1'],
 'C2': ['F1', 'D1', 'C2'],
 'C3': ['F1', 'D1', 'C3'],
 'C4': ['F1', 'C4'],
 'C5': ['F1', 'D3', 'C5'],
 'C6': ['F1', 'C6']}

In [7]:
#若从F2工厂发出,若不经过D2或D4时，F2到达不了C4，此时只能由F1到达C4
path2={}
for i in [1,2,3,5,6]:
    path2['C'+str(i)]=nx.dijkstra_path(FG, source='F2', target="C"+str(i), weight='cost')
path2

{'C1': ['F2', 'C1'],
 'C2': ['F2', 'D3', 'C2'],
 'C3': ['F2', 'D3', 'C3'],
 'C5': ['F2', 'D3', 'C5'],
 'C6': ['F2', 'D3', 'C6']}

In [8]:
#从F1发出时，到达各客户最优费用
import collections
origin=data['origin']
dest=data['dest']
weight=data['cost']
fee=collections.defaultdict(dict)
best_fee1=collections.defaultdict(int)
for i in range(len(origin)):
    fee[origin[i]][dest[i]]=weight[i]
for i in range(1,7):
    sig=path1['C'+str(i)]
    length=len(sig)
    for j in range(length-1):
        best_fee1['C'+str(i)]+=fee[sig[j]][sig[j+1]]
best_fee1

defaultdict(int,
            {'C1': 1.0, 'C2': 2.0, 'C3': 1.0, 'C4': 2.0, 'C5': 1.5, 'C6': 1.0})

In [9]:
#从F2发出时，到达各客户最优费用
best_fee2=collections.defaultdict(int)
for i in range(len(origin)):
    fee[origin[i]][dest[i]]=weight[i]
for i in [1,2,3,5,6]:
    sig=path2['C'+str(i)]
    length=len(sig)
    for j in range(length-1):
        best_fee2['C'+str(i)]+=fee[sig[j]][sig[j+1]]
best_fee2

defaultdict(int, {'C1': 2.0, 'C2': 2.0, 'C3': 2.5, 'C5': 1.0, 'C6': 2.0})

In [10]:
fee_diff=collections.defaultdict(int)
for i in [1,2,3,5,6]:
    fee_diff['C'+str(i)]=best_fee1['C'+str(i)]-best_fee2['C'+str(i)]
fee_diff


defaultdict(int, {'C1': -1.0, 'C2': 0.0, 'C3': -1.5, 'C5': 0.5, 'C6': -1.0})

综合最佳路径时的分配方式,到达C1,C6的路径不变，F1-C6 20000吨；F1-C1 50000吨。需要调整的是运往C2或C4，以及运往C3或C5的路径，这之中根据从F1,F2发出时价格差值的大小决定考虑的优先顺序。对于C4，由于只能由F1到达，只能由F1来发往C4。其他的消费者可以这样分配：F1-D1-C3 40000吨；F2-D3-C5 60000吨；F2-D3-C2 10000吨

这是完全不考虑经过D2,D4仓库的情况，现在我们再把D2,D4仓库考虑进来，这个时候，在满足D2,D4最大吞吐量的情况下，我们考虑经过D2,D4仓库时，分别从F1,F2发出的最优与次优路径的差值，根据差值大小来决定调整发往C3,C5,C2的分配。


对F1:最优时：F1-D2-C2 1；F1-D4-C3 0.4；F1-D2-C4 1.5；F1-D4-C5 0.7次优时：F1-D1-C2 2；F1-D1-C3 1；F1-C4 2；F1-D3-C5 1.5，差值为：1，0.6，0.5，0.8

对于F2：最优时：F2-D2-C2 0.8；F2-D4-C3 0.4； F2-D4-C5 0.7次优时：F2-D3-C2 2；F2-D3-C3 2.5；F2-D3-C5 1.0，差值为：1.2,2.1,0.3

考虑最终方案：
1.到达C1,C6的路径不变，F1-C6 20000吨；F1-C1 50000吨
2.由F1-C4 35000吨，F1还剩45000吨
3.剩下之中，再考虑最佳分配：可以发现由F2-D4-C3的最优与次优价格相差最大，因而尽量满足其最优，F2-D4-C3 20000吨，D4已满，将其移去
4.剩下之中，F2-D2-C2的最优与次优价格相差最大，因而尽量满足其最优，F2-D2-C2 10000吨 D2还剩15000吨容量
5.C5还剩60000吨，F2-D3-C5最便宜，但是D3的吞吐量只有100000吨，只能中转50000吨F2-D3-C5，还剩10000吨要转到C5，且这10000吨只能通过仓库转运
6.要先满足C5,F2-D2-C5 10000吨，D2已满
7.D2,D4已满，剩下之中考虑F1与F2次优解中价格差最大的，价格差最大为到C3的次优解，应尽量安排F1-D1-C3 35000吨，F1还剩10000吨，D1已满
8.剩下的5000吨C3路径：F1-C3



因此，满足条件的最终方案为：
F1-C1 50000吨；F2-D2-C2 10000吨；F1-D1-C3 35000吨；F1-C3 5000吨；F1-C4 35000吨；F2-D3-C5 50000吨；F2-D2-C5 10000吨；F1-C6 20000吨
总价格为：248500元


# 如果增加工厂和 DC 仓库的面积，对物流成本会有如何影响？

可以看到C2-C5受到仓库吞吐量不足的限制，使得运输成本大大提高，因而如果可以增加仓库的面积，总体来说对于物流成本的降低十分有利

# 运输成本，仓库面积和经销商的采购量的变化会对发送模式有什么影响？

1.运输成本对发运模式的影响：当运输成本较高时，对C2-C5来说，发运模式可能更多的采用直接从工厂运到经销商，对C1,C6来说，发运模式可能是经过仓库中转的模式；当运输成本较低时，则C2-C5更多采用经过仓库中转的方式，发送到C1,C6的更多的采用直接从工厂发送的方式。

2.仓库面积对发运模式的影响：在之前的分析中，可以发现，由于D2,D4仓库的吞吐量不足，限制了运输到C2-C5经销商的最优路径的实施，所以当这两个仓库的面积增加时，可以更多的采用最优的发运模式，发运时更多的经过仓库的中转从而减小C2-C5成本，从而减少总成本；而当仓库面积变小时，发送到C2-C5的货物则只能采用成本相对较高的发运模式，更多的是不经过仓库，而是直接从工厂运往经销商的发运模式。

3.经销商采购量对发运模式的影响：采购量小时，对于仓库中转的压力小，可以更多采用最优的发运模式；采购量大时，受到仓库面积的限制，只能更多采取次优的发运模式，成本也会相应的高。

# 如果经销商希望从他们指定的工厂或 DC 买货，而且我们去满足他们的偏好，会给我们增加多少物流成本？

（由于要满足偏好，仓库的吞吐量势必不够，比如要满足C5来自D2，所以我理解为这个时候是不需要考虑仓库容量的限制的，那么之后计算增加多少成本时，我会将结果和最优的方案进行比较）
满足偏好时：
C1来自F1：路径 F1-C1 50000吨 价格 1x50000
C2来自D1仓库：路径 F1-D1-C2 10000吨 价格2x10000
C3都可以：路径 F2-D4-C3 40000吨 价格 0.4x40000
C4都可以：路径 F2-D2-C4 35000吨 价格 1.3x35000
C5来自D2：路径 F1-D4-C5 60000吨 价格 0.7x60000
C6 D3或者D4：路径 F2-D3-C6 20000吨 价格 1.7x20000
总物流成本：207500元
增加了（207500-181500）=26000元成本